In [3]:
import pandas as pd
import requests
import datetime
import pyarrow
import duckdb
from io import StringIO

load_date = datetime.date.today()   

response = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2025-12-15&endtime=2026-01-15")

response.raise_for_status()
    
csv_text = response.text

In [4]:
df = pd.read_csv(StringIO(csv_text))


In [42]:
df.head(10)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2026-01-14T23:49:34.240Z,38.832333,-122.819168,1.7700,0.76,md,13,57.0,0.014190,0.01,...,2026-01-14T23:51:11.631Z,"8 km NW of The Geysers, CA",earthquake,0.310000,0.710000,0.530000,14.0,automatic,nc,nc
1,2026-01-14T23:44:57.920Z,53.894333,-166.625333,7.8300,-0.44,ml,6,148.0,0.037470,0.13,...,2026-01-15T09:48:21.520Z,"5 km W of Dutch Harbor, Alaska",earthquake,0.390000,0.790000,0.181362,6.0,reviewed,av,av
2,2026-01-14T23:42:22.140Z,35.344000,-117.810167,6.9500,1.35,ml,35,36.0,0.081780,0.19,...,2026-01-15T15:32:58.068Z,"16 km WSW of Johannesburg, CA",earthquake,0.210000,0.510000,0.211954,26.0,reviewed,ci,ci
3,2026-01-14T23:40:59.760Z,38.813835,-122.795334,1.1500,0.38,md,8,66.0,0.005859,0.01,...,2026-01-14T23:42:37.715Z,"5 km NW of The Geysers, CA",earthquake,0.460000,1.010000,0.180000,8.0,automatic,nc,nc
4,2026-01-14T23:38:19.392Z,-21.360400,170.221200,110.9190,4.90,mww,72,33.0,2.861000,1.05,...,2026-01-15T00:17:09.040Z,"223 km SSE of Isangel, Vanuatu",earthquake,8.980000,4.800000,0.086000,13.0,reviewed,us,us
5,2026-01-14T23:34:39.409Z,61.599000,-150.456000,53.6000,1.90,ml,25,50.0,0.500000,0.40,...,2026-01-14T23:36:39.944Z,"6 km NNE of Susitna, Alaska",earthquake,3.500000,6.533600,0.400000,3.0,automatic,ak,ak
6,2026-01-14T23:33:49.639Z,32.363000,-102.057000,7.9346,0.50,ml,10,82.0,0.000000,0.10,...,2026-01-15T14:56:20.245Z,"36 km NW of Stanton, Texas",earthquake,0.906366,0.863904,0.200000,6.0,reviewed,tx,tx
7,2026-01-14T23:32:21.190Z,18.116333,-66.153667,25.6200,1.88,md,13,98.0,0.007995,0.11,...,2026-01-14T23:45:05.840Z,"1 km ENE of Cayey, Puerto Rico",earthquake,0.410000,0.550000,0.078936,4.0,reviewed,pr,pr
8,2026-01-14T23:28:32.130Z,18.109833,-66.142667,25.5100,1.75,md,8,99.0,0.007013,0.05,...,2026-01-14T23:47:27.710Z,"2 km E of Cayey, Puerto Rico",earthquake,0.380000,0.480000,0.068359,6.0,reviewed,pr,pr
9,2026-01-14T23:21:29.240Z,33.946667,-116.695333,11.7100,1.01,ml,36,48.0,0.098180,0.17,...,2026-01-14T23:25:10.457Z,"9 km ENE of Cabazon, CA",earthquake,0.190000,0.600000,0.136823,26.0,automatic,ci,ci


In [28]:
df.to_csv("eatherquake_full.csv", index=False)

In [30]:
df.to_parquet("eatherquake_full.parquet", index=False)

In [5]:
a = pd.DataFrame({
        "time" : df["time"],
        "latitude" : df["latitude"],
        "mag" : df["mag"],
        "place" : df["place"],
        "updated" : df["updated"],
    
})

a.head(5)

,time,latitude,mag,place,updated
0,2026-01-14T23:49:34.240Z,38.832333,0.76,"8 km NW of The Geysers, CA",2026-01-14T23:51:11.631Z
1,2026-01-14T23:44:57.920Z,53.894333,-0.44,"5 km W of Dutch Harbor, Alaska",2026-01-15T09:48:21.520Z
2,2026-01-14T23:42:22.140Z,35.344000,1.35,"16 km WSW of Johannesburg, CA",2026-01-15T15:32:58.068Z
3,2026-01-14T23:40:59.760Z,38.813835,0.38,"5 km NW of The Geysers, CA",2026-01-14T23:42:37.715Z
4,2026-01-14T23:38:19.392Z,-21.360400,4.90,"223 km SSE of Isangel, Vanuatu",2026-01-15T00:17:09.040Z


In [6]:
a.to_parquet("eatherquake_clear.parquet", index=False)

In [7]:
cursor = duckdb.connect()
print(cursor.execute('SELECT 42').fetchall())

[(42,)]


In [11]:
cursor.sql(
    """
    SELECT * FROM read_parquet('Api_lesson/eatherquake_clear.parquet')
    """
).df()

,time,latitude,mag,place,updated
0,2026-01-14T23:49:34.240Z,38.832333,0.76,"8 km NW of The Geysers, CA",2026-01-14T23:51:11.631Z
1,2026-01-14T23:44:57.920Z,53.894333,-0.44,"5 km W of Dutch Harbor, Alaska",2026-01-15T09:48:21.520Z
2,2026-01-14T23:42:22.140Z,35.344000,1.35,"16 km WSW of Johannesburg, CA",2026-01-15T15:32:58.068Z
3,2026-01-14T23:40:59.760Z,38.813835,0.38,"5 km NW of The Geysers, CA",2026-01-14T23:42:37.715Z
4,2026-01-14T23:38:19.392Z,-21.360400,4.90,"223 km SSE of Isangel, Vanuatu",2026-01-15T00:17:09.040Z
...,...,...,...,...,...
8948,2025-12-15T00:07:09.370Z,38.362333,2.12,"5 km W of Glen Ellen, CA",2025-12-26T17:17:21.978Z
8949,2025-12-15T00:04:20.880Z,38.379333,3.33,"6 km SW of Kenwood, CA",2026-01-01T23:45:28.040Z
8950,2025-12-15T00:03:51.802Z,31.936000,2.00,"27 km SW of Jal, New Mexico",2025-12-15T14:25:44.938Z
8951,2025-12-15T00:02:50.668Z,60.576000,2.20,"115 km N of Yakutat, Alaska",2026-01-12T23:31:22.924Z


In [18]:
cursor.sql(
    """
    SELECT *
    FROM read_parquet('Api_lesson/eatherquake_clear.parquet')
    WHERE place like '%Alaska%'
    """
).df()

,time,latitude,mag,place,updated
0,2026-01-14T23:44:57.920Z,53.894333,-0.44,"5 km W of Dutch Harbor, Alaska",2026-01-15T09:48:21.520Z
1,2026-01-14T23:34:39.409Z,61.599000,1.90,"6 km NNE of Susitna, Alaska",2026-01-14T23:36:39.944Z
2,2026-01-14T23:12:16.036Z,61.344000,1.40,"47 km WNW of Beluga, Alaska",2026-01-14T23:16:10.851Z
3,2026-01-14T22:22:09.024Z,60.068000,1.70,"10 km W of Ninilchik, Alaska",2026-01-14T22:24:59.858Z
4,2026-01-14T22:21:13.458Z,56.063000,3.10,"44 km SE of Chignik, Alaska",2026-01-14T22:23:20.852Z
...,...,...,...,...,...
2571,2025-12-15T00:26:44.523Z,63.154000,1.80,"63 km ESE of Cantwell, Alaska",2026-01-13T01:57:32.138Z
2572,2025-12-15T00:12:54.253Z,60.572000,2.20,"114 km N of Yakutat, Alaska",2026-01-12T23:50:38.236Z
2573,2025-12-15T00:11:34.550Z,64.967000,1.50,"10 km E of Fox, Alaska",2026-01-12T23:35:53.689Z
2574,2025-12-15T00:02:50.668Z,60.576000,2.20,"115 km N of Yakutat, Alaska",2026-01-12T23:31:22.924Z
